In [0]:
!pip3 install epitran

In [123]:
# import epitran
import pandas as pd
import numpy as np
import re
import networkx as nx
import sklearn
import pickle

In [0]:
epi = epitran.Epitran('rus-Cyrl')
epi

In [0]:
epi.transliterate(u'мамулечка')

'mamulʲet͡ɕʲka'

In [51]:
with open ('words.csv', 'r', encoding = 'utf-8') as f:
    file = f.read().replace('?', '')
with open ('rules.csv', 'r', encoding = 'utf-8') as r:
    rules = r.read()

In [52]:
rules = rules.split('\n')
list_words = file.split(';\n')

# print(rules)
# print(list_words)

In [53]:
# ПОСТРОЕНИЕ ТРАНСКРИПЦИЙ В ЭПИТРАНЕ

# true_list_words = []
# list_words_ipa = []

# for two_words in list_words:
#     two_words = two_words.split(';')
#     true_list_words.append(two_words)
#     for word in two_words: 
#         word_ipa = epi.transliterate(word)
#         list_words_ipa.append(word_ipa)
# list_words_ipa

NameError: name 'epi' is not defined

In [54]:
!pip install phonetic-algorithmIPA
from phonetic_algorithmIPA import ipa_distances
ipa = ipa_distances.PhoneticAlgorithmIPA()

In [130]:
# ПОСТРОЕНИЕ ТРАНСКРИПЦИЯ В phonetic-algorithmIPAБ
# ПОИСК РАССТОЯНИЯ ЛЕВЕНШТЕЙНА

# a = ipa.phonetic_transformer('words.csv', 'rules.csv', typ='Non LS')
# a

In [30]:
list_rules = []
for i in rules:
    list_rules.append(i.split(';'))

In [31]:
word = 'меня'

def zamena(word, index_letter, rule):
    # print(index_letter)
    letter = word[index_letter]
    if rule[0] == word[index_letter]:
        if rule[2] == '':
            word = word.replace(word[index_letter], rule[1])
            # index_letter += len(rule[1]) - len(rule[0])
            return word

        if rule[2][1] == word[index_letter+1] or rule[2][0] == word[index_letter-1]:
            if rule[2][1] == word[index_letter+1]:
                word = word.replace(word[index_letter], rule[1], 1)
                # index_letter += len(rule[1]) - len(rule[0])
                return word

        # print(rule[2][0] == word[index_letter-1])
        if rule[2][0] == word[index_letter-1]:
            print(rule[2][0])
            word = word.replace(word[index_letter], rule[1], 1)
            # index_letter += len(rule[1]) - len(rule[0])
            return word

        else:
            # print('e')
            # index_letter += len(rule[1]) - len(rule[0])
            return word

    else:
    # index_letter += len(rule[1]) - len(rule[0])
        return word


In [32]:
# РАЗБИЕНИЕ ПРАВИЛ ПО ПРИОРИТЕТАМ ПРИМЕНЕНИЯ К СЛОВУ

letter = 'м'
def rule_for_letter(letter, rules):
    firstPrior_rules_for_letter = []
    SecondPrior_rules_for_letter = []
    for rule in rules:
        if rule[0] == letter and rule[2] != '':
            firstPrior_rules_for_letter.append(rule)
            # print('prior', firstPrior_rules_for_letter)
        elif rule[0] == letter and rule[2] == '':
            SecondPrior_rules_for_letter.append(rule)
            # print('second', SecondPrior_rules_for_letter)
    return firstPrior_rules_for_letter + SecondPrior_rules_for_letter

ru = rule_for_letter(letter, list_rules)
ru

[['м', 'мʲ', '_j'],
 ['м', 'мʲ', '_е'],
 ['м', 'мʲ', '_ё'],
 ['м', 'мʲ', '_и'],
 ['м', 'мʲ', '_ю'],
 ['м', 'мʲ', '_я'],
 ['м', 'm', '']]

In [33]:
# ФУНКЦИЯ ДЕЛАЕТ ТРАНСКРИПЦИЮ

def make_trans(word):
    word_base = word
    index_letter = 0
    for letter in word:
        prioritete_rule = rule_for_letter(letter, list_rules)
        # print(prioritete_rule, word)
        for i in prioritete_rule:
            # print(word, index_letter, i)
            word = zamena(word, index_letter, i)
        index_letter += 1 + len(word) - len(word_base)
        word_base = word
        # print(word)
    return word

make_trans('взирает#')

'vzʲiraet#'

In [58]:
short_list = []
for two_w in list_words:
    j = two_w.split(';')
    short_list.append(j)
short_list[-2]

['домой', 'зимой']

In [129]:
big_list_trans = []
for pair in short_list[:-1]:
    for word in pair:
        two_w = make_trans(word +'#')
        big_list_trans.append(two_w)


In [69]:
#  with open('transcript.csv', 'w', encoding = 'utf-8') as f:
#   for pair in big_list_trans:
#     for w in pair:
#       f.write(w)
#     f.write('\n')
# with open('transcript.csv', 'r', encoding = 'utf-8') as f:
#   fi = f.read()
#   print(fi)

In [78]:
# СОЕДИНЯЕТ ДИАКРИТИКИ И ДРУГИЕ СПЕЦИАЛЬНЫЕ СО ВЛОВАМИ, К КОТОРЫМ ОНИ ОТНОСЯТСЯ

diacritic = { "ʲ" : [ 'pos', { 'round': '+' } ], "ˈ" : [ 'pre', { 'round': '+' } ],  ")": [ 'betw', { 'round': '+' } ]}
def find_diacr(word, letter, symbol):
    word = list(word)
    if symbol[0] == letter:
        if diacritic[symbol][0] == 'pos':
            pre = word[:word.index(letter) - 1]
            diacr = [word[word.index(letter) - 1] + word[word.index(letter)]]
            post = word[word.index(letter) + 1:]
            new_word = pre + diacr + post
            # print(new_word)
            return new_word, letter

# diacritic[symbol][0]
        elif diacritic[symbol][0] == 'pre':
            pre = word[:word.index(letter)]
            diacr = [word[word.index(letter)] + word[word.index(letter)+ 1 ]]
            post = word[word.index(letter) + 2:]
            new_word = pre + diacr + post
            letter = word[word.index(letter) + 2]
            # print(letter)
            return new_word, letter 
      
        elif diacritic[symbol][0] == 'betw':
            pre = word[:word.index(letter) - 1]
            diacr = [word[word.index(letter) - 1 ] + word[word.index(letter)] + word[word.index(letter)+ 1]]
            post = word[word.index(letter) + 2:]
            new_word = pre + diacr + post
            letter = word[word.index(letter) + 2]
            # print(new_word)
            return new_word, letter 

    else:
        return word, letter
    

print(find_diacr (['m', ')', 'e', 'n', 'a'], ')', ')'))

(['m)e', 'n', 'a'], 'n')


In [63]:
# РАЗДЕЛЯЕТ ВСЕ СЛОВА В ЛИСТЕ НА СЕГМЕНТЫ

def make_segments_list(big_list_trans):
    segments = []
    for word in big_list_trans:
        try:
            for letter in word:
                for symbol in diacritic.keys():
                    word, letter = find_diacr(word, letter, symbol)
        except ValueError:
            pass
        segments.append(word)
    return segments

In [68]:
with open('puschkin.txt', 'r', encoding = 'utf-8') as f:
    file = f.read()
    file = re.sub(r"[,!?.;:\-\'\"]", '', file)
file = file.split('\n')
print(file)

['Духовной жаждою томим', 'В пустыне мрачной я влачился ', 'И шестикрылый серафим', 'На перепутьи мне явился', 'Перстами легкими как сон', 'Моих зениц коснулся он', 'Отверзлись вещие зеницы', 'Как у испуганной орлицы', 'Моих ушей коснулся он ', 'И их наполнил шум и звон', 'И внял я неба содроганье', 'И горний ангелов полет', 'И гад морских подводный ход', 'И дольней лозы прозябанье', 'И он к устам моим приник', 'И вырвал грешный мой язык', 'И празднословный и лукавый', 'И жало мудрыя змеи', 'В уста замершие мои', 'Вложил десницею кровавой', 'И он мне грудь рассек мечом', 'И сердце трепетное вынул', 'И угль пылающий огнем', 'Во грудь отверстую водвинул', 'Как труп в пустыне я лежал', 'И бога глас ко мне воззвал', 'Восстань пророк и виждь и внемли', 'Исполнись волею моей', 'И обходя моря и земли', 'Глаголом жги сердца людей']


In [131]:
# ДОСТАЕТ В СТИХЕ ПУШКИНА ПОСЛЕДНИЕ СЛОВА

list_word_to_transc = []
for i in file:
    # print(i)
    if i != '':
        last_word = i.rstrip(' ').split(' ')[-1]
        print(last_word)
        if len(last_word) >= 4:
            word_to_transc = last_word
            list_word_to_transc.append(word_to_transc)
        else:
            word_to_transc = i.split(' ')[-2:]
            word_to_transc = ' '.join(word_to_transc)
            list_word_to_transc.append(word_to_transc)
    else:
        file.remove('')


# list_word_to_transc

томим
влачился
серафим
явился
сон
он
зеницы
орлицы
он
звон
содроганье
полет
ход
прозябанье
приник
язык
лукавый
змеи
мои
кровавой
мечом
вынул
огнем
водвинул
лежал
воззвал
внемли
моей
земли
людей


In [81]:
# ДЕЛАЕТ ТРАНСКРИПЦИЮ ДЛЯ СПИСКА СЛОВ

def make_trans_for_list(list_word_to_transc):
    list_transc_push = []
    for i in list_word_to_transc:
        if ' ' in i:
            two_words_in_one = i.replace(' ' , '')
            list_transc_push.append(make_trans(two_words_in_one + '#'))
        else:
            list_transc_push.append(make_trans(i+'#'))
    return list_transc_push

In [80]:
segments = make_segments_list(make_trans_for_list(list_word_to_transc))
segments

[['t', 'o', 'mʲ', 'i', 'm', '#'],
 ['v', 'l', 'a', 't)ɕʲ', 'i', 'l', 'sʲ', 'a', '#'],
 ['sʲ', 'e', 'r', 'a', 'fʲ', 'i', 'm', '#'],
 ['a', 'vʲ', 'i', 'l', 's', 'a', '#'],
 ['k', 'a', 'k', 's', 'o', 'n', '#'],
 ['k', 'o', 's', 'n', 'u', 'l', 's', 'a', 'o', 'n', '#'],
 ['zʲ', 'e', 'nʲ', 'i', 't)s', 'ɨ', '#'],
 ['o', 'r', 'lʲ', 'i', 't)s', 'ɨ', '#'],
 ['o', 'n', '#'],
 ['z', 'v', 'o', 'n', '#'],
 ['s', 'o', 'd', 'r', 'o', 'ɡ', 'a', 'n', 'e', '#'],
 ['p', 'o', 'lʲ', 'e', 't', '#'],
 ['p', 'o', 'd', 'v', 'o', 'd', 'n', 'ɨ', 'j', 'x', 'o', 'd', '#'],
 ['p', 'r', 'o', 'zʲ', 'a', 'b', 'a', 'n', 'e', '#'],
 ['p', 'rʲ', 'i', 'n', 'i', 'k', '#'],
 ['a', 'z', 'ɨ', 'k', '#'],
 ['l', 'u', 'k', 'a', 'v', 'ɨ', 'j', '#'],
 ['z', 'mʲ', 'e', 'i', '#'],
 ['z', 'a', 'mʲ', 'e', 'r', 'ʂ', 'i', 'e', 'm', 'o', 'i', '#'],
 ['k', 'r', 'o', 'v', 'a', 'v', 'o', 'j', '#'],
 ['mʲ', 'e', 't)ɕʲ', 'o', 'm', '#'],
 ['v', 'ɨ', 'n', 'u', 'l', '#'],
 ['o', 'ɡ', 'nʲ', 'e', 'm', '#'],
 ['v', 'o', 'd', 'v', 'i', 'n', 'u', 'l',

In [82]:
# ВЫДЕЛЯЕТ ПОСЛЕДНИЕ ЧЕТЫРЕ ЭЛЕМЕНТА В СЛОВЕ


def segments_four_last_func(segments):
    segments_four_last = ''
    count = 0
    for i in segments:
        str_word = ''
        str_word = ''.join(i[-5:-1:])
        if count % 2 == 0:
            segments_four_last += ''.join(str_word + ';')
            count += 1
        else:
            segments_four_last += ''.join(str_word + '\n')
            count += 1
    print(segments_four_last)
    return segments_four_last

In [91]:
# СЧИТАЕТ РАССТОЯНИЕ ПО ЛЕВЕНШТЕЙНУ МЕЖДУ ДВУМЯ СЛОВАМИ


def find_distance(str_word_first, str_word_second):
    with open('trans_four.csv', 'w', encoding = 'utf-8') as f:
        f.write(str_word_first + ';' + str_word_second)
    c = ipa.phonetic_distance('trans_four.csv', normalize = False)
    return c

In [89]:
# ФУНКЦИЯ ИЩЕТ ВОСЕМЬ БЛИЖАЙШИХ СТРОК К ЗАДАНОЙ СТРОКЕ


def find_eight_string(list_words, startword):
    if list_words.index(startword) == 0 or list_words.index(startword) == 1 or list_words.index(startword) == 2 or list_words.index(startword) == 3:
        befor = list_words[: list_words.index(startword)]
    else:
        befor = list_words[list_words.index(startword) -4  : list_words.index(startword)]
    after = list_words[list_words.index(startword) + 1 : list_words.index(startword) + 5]
    return befor, after

# ФУНКЦИЯ ВЫДАЕТ ДИСТАНЦИЮ МЕЖДУ СТАРТОВЫМ УЗЛОМ И УЗЛАМИ, СВЯЗАННЫМИ С НИМ  


def make_corteg_distance_for_nodes(startword, befor, after):
    list_corteg_distance_for_nodes = []
    startword_seg = make_segments_list(make_trans_for_list([startword]))[0]
    str_word_first = ''.join(startword_seg[-4:-1:])
    if befor != []:
        seg_befor = make_segments_list(make_trans_for_list(befor))
        for i in seg_befor:
            str_word_second = ''.join(i[-4:-1:])
            cort = (startword, befor[seg_befor.index(i)], find_distance(str_word_first, str_word_second))
            list_corteg_distance_for_nodes.append(cort)

    if after != []:
        seg_after = make_segments_list(make_trans_for_list(after))
        for i in seg_after:
            str_word_second = ''.join(i[-4:-1:])
            cort = (startword, after[seg_after.index(i)], find_distance(str_word_first, str_word_second))
            list_corteg_distance_for_nodes.append(cort)
    return list_corteg_distance_for_nodes
  

  




# лист восьми (спереди и сзади) ближайших соседей слова старт
# построение их транскрипции 
# нахождение растоянияния для каждой
# 
# записываю новый узел в граф если узел = минимальное растояние до этого слова
# подумать как оформить одиночные строки

In [85]:
def find_min(node):
    weight =None
    min_dist = 3
    cort_min = ()
    for j in G.edges(node):
        weight = G.get_edge_data(j[0], j[1])['weight']
        if min_dist == 3:
            min_dist = weight
            cort_min = (min_dist, j)

        elif min_dist > weight:
            New_G.add_edge(j[0], j[1])
            min_dist = weight
            cort_min = (min_dist, j)
    return cort_min 

In [135]:
# ПЫТАЮСЬ ПОСТРОИТЬ ГРАФ (New_G), СВЯЗИ В КОТОРОМ БУДУТ ОБОЗНАЧАТЬ РИФМУ, А УЗЛЯ - РИФМУЮЩИЕСЯ СЛОВА


weight =None
min_dist = 3  
list_cort_edges_for_G = []

New_G = nx.Graph()
for i in list_word_to_transc:
    New_G.add_node(i)
    
for i in list_word_to_transc:
    G=nx.Graph()
    if type(i) == list:
        i = ' '.join(i)
    befor_startword, after_startword = find_eight_string(list_word_to_transc, i)
    list_corteg_distance_for_node_i = make_corteg_distance_for_nodes(i, befor_startword, after_startword)
    G.add_weighted_edges_from(list_corteg_distance_for_node_i)
#     print(G.edges())
    cort_min = find_min(i)
#     print(cort_min, '\n', cort_min[1][0], cort_min[1][1])
    list_cort_edges_for_G.append((cort_min[1][0], cort_min[1][1]))
print(list_cort_edges_for_G)
New_G.add_edges_from(list_cort_edges_for_G)


[('томим', 'серафим'), ('влачился', 'явился'), ('серафим', 'томим'), ('явился', 'влачился'), ('как сон', 'коснулся он'), ('коснулся он', 'звон'), ('зеницы', 'орлицы'), ('орлицы', 'зеницы'), ('он ', 'звон'), ('звон', 'подводный ход'), ('содроганье', 'прозябанье'), ('полет', 'приник'), ('подводный ход', 'звон'), ('прозябанье', 'содроганье'), ('приник', 'язык'), ('язык', 'приник'), ('лукавый', 'кровавой'), ('змеи', 'замершие мои'), ('замершие мои', 'змеи'), ('кровавой', 'лукавый'), ('мечом', 'огнем'), ('вынул', 'водвинул'), ('огнем', 'мечом'), ('водвинул', 'вынул'), ('лежал', 'воззвал'), ('воззвал', 'лежал'), ('внемли', 'земли'), ('моей', 'людей'), ('земли', 'внемли'), ('людей', 'моей')]


In [93]:
pos = nx.spring_layout(New_G)
nx.draw(New_G, pos, with_labels=True)

In [94]:
# ДАЛЬШЕ Я НАЧИНАЮ ПОДГОТОВКУ ДАННЫХ К ОБУЧЕНИЮ СВОЕЙ МОДЕЛЬКИ

In [98]:
with open ('priznaki.txt', 'r') as f:
    priznaki = f.readlines()

In [125]:
# ПРИПИСЫВАЕТ ПРИЗНАКИ ЗВУКАМ, ВЫДАЕТ ЛИСТ С ЛИСТАМИ

list_priznaki = []
for i in priznaki:
    list_priznaki.append(i.split(';'))

def create_matrix_priz(list_priznaki, word):
    list_priz_for_word = []
    for letter in word[-4:-1:]:
        for i in list_priznaki:
            if letter == i[0]:
                list_priz_for_word.append(i[2:17])

#     matrix_priz_for_word = np.array(a)
    return list_priz_for_word

# for word in segments:
  # print(word)
print(create_matrix_priz(list_priznaki, ['mʲ', 'e', 'nʲ', 'a', '#']))

[['+', '+', '-', '?', '?', '-', '-', '-', '-', '+', '?', '?', '?', '?', '?'], ['+', '+', '-', '?', '?', '-', '+', '+', '-', '+', '?', '?', '?', '?', '?']]


In [100]:
simple = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
len(simple)

15

In [101]:
# ДЕЛИТ СЛОВО НА СЕГМЕНТЫ

def make_segments_for_word(word):
    for letter in word:
        for symbol in diacritic.keys():
            word, letter = find_diacr(word, letter, symbol)
        # print(word)
    return word
make_segments_for_word('dnʲa#')

['d', 'nʲ', 'a', '#']

In [104]:
# ФУНКЦИЯ ВОЗВРАЩАЕТ ВЕКТОР В КОТОРОМ ЕДИНИЦАМИ УКАЗАНЫ ОБЩИЕ ЭЛЕМЕНТЫ, А НУЛЯМИ РАЗЛИЧНЫЕ

def matr_egal(matr1, matr2):
    list_egalite = []
    for i in range(len(matr1)):
        for j in range(len(matr1[i])):
            if matr1[i][j] == matr2[i][j]:
                list_egalite.append(1)
            else:
                list_egalite.append(0)
    return list_egalite

matr1 = [['-', '-', '-', '?', '+', '?', '-', '+', '?', '-', '-', '-', '-', '-', '+'], ['+', '+', '-', '?', '?', '-', '+', '+', '-', '+', '?', '?', '?', '?', '?']]

matr2 =  [['-', '-', '+', '+', '+', '?', '-', '-', '?', '-', '-', '-', '-', '-', '+'], ['+', '+', '-', '?', '?', '-', '+', '+', '-', '+', '?', '?', '?', '?', '?']]

c = matr_egal(matr1, matr2)
print(c)

[1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [136]:
dataf = pd.read_csv('data_vectots.csv')

In [111]:
print(dataf.head())
dataf1 = np.copy(dataf)

   Unnamed: 0  0  1  2  3  4  5  6  7  8 ...  36  37  38  39  40  41  42  43  \
0          40  1  1  1  1  0  1  1  1  1 ...   1   1   1   1   1   1   1   1   
1          43  1  0  1  1  1  1  1  1  1 ...   1   1   1   1   1   1   1   1   
2          45  1  1  1  1  1  1  0  1  0 ...   1   1   1   1   1   1   1   1   
3          48  1  1  1  1  1  1  1  1  1 ...   1   1   1   1   1   1   1   1   
4          60  1  1  1  1  1  1  0  1  0 ...   1   1   1   1   1   1   1   1   

   44  45  
0   1   1  
1   0   1  
2   1   1  
3   1   1  
4   1   1  

[5 rows x 47 columns]


In [122]:
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import datasets

ImportError: No module named 'sklearn.model_selection'

In [116]:
np.random.shuffle(dataf1)

In [117]:
np.random.shuffle(dataf1)
len(dataf1)
train = dataf1[:70000]
test = dataf1[70000:]

X_train = train[:,:45]
Y_train = train[ :,45]

X_test = test[:,:45]
Y_test = test[ :,45]
print(X_train, Y_train)

[[3469    1    1 ...,    1    1    1]
 [3661    1    1 ...,    1    1    1]
 [2454    0    1 ...,    1    1    1]
 ..., 
 [ 383    0    0 ...,    1    1    1]
 [  43    1    0 ...,    1    1    1]
 [ 823    0    0 ...,    1    1    1]] [1 1 1 ..., 1 0 1]


In [ ]:
ridge = RidgeClassifier()
alphas = np.array([1, 0.1, 0.01, 0.001, 0.0001, 0])
parameters = dict(alpha = alphas)

clf = GridSearchCV(ridge, parameters, cv=5)
clf.fit(X_train, Y_train)

In [0]:
print(clf.best_score_)
print(clf.best_params_)
print(clf.best_estimator_)
print(clf.best_estimator_.alpha)

0.9186
{'alpha': 1.0}
RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)
1.0


In [0]:
answers = clf.best_estimator_.predict(X_test)
print(Y_test[:20])

['0' '0' '1' '1' '0' '1' '0' '1' '0' '0' '0' '1' '1' '1' '0' '0' '0' '0'
 '1' '1']


In [0]:
model = clf.best_estimator_
pickle.dump(model, open('model.sav', 'wb'))

In [0]:
nonr = np.array(nonr).reshape(1, -1)
print(X_test)
answers = clf.best_estimator_.predict(nonr)
print(answers)

[[0 0 0 ... 0 1 1]
 [1 1 0 ... 1 1 1]
 [1 0 0 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [0 1 1 ... 0 1 1]]
['1']


In [0]:
(answers == Y_test).astype(int).mean()

0.9182763196697139

In [126]:
# ПРОГА СОЗДАЕТ ВЕКТОР ПОХОЖЕСТИ ДВУХ ВЕКТОРОВ, КОТОРЫЙ ПОДАЕТСЯ НА ВХОД МОДЕЛИ

tr = make_trans('мурзилка#')

tr2 = make_trans('корзинка#')

tr_seg = make_segments_for_word(tr)
tr_seg2 = make_segments_for_word(tr2)

create_matrix_priz(list_priznaki, make_segments_for_word(tr))
create_matrix_priz(list_priznaki, make_segments_for_word(tr2))

nonr = matr_egal(create_matrix_priz(list_priznaki, make_segments_for_word(tr)), create_matrix_priz(list_priznaki, make_segments_for_word(tr2)))
nonr = np.array(nonr).reshape(1, -1)

In [ ]:
load_model = pickle.load(open('model.sav', 'rb'))

In [ ]:
# ДАЕТ ОТВЕТ РИФМУЮТСЯ ИЛИ НЕТ ДВА СЛОВА, НА ВХОД ПОЛУЧАЕТ ИХ ВЕКТОР СХОЖЕСТИ

load_model.predict(nonr)

In [137]:
# ОСНОВНОЙ КОД, ДЕЛАЕТ БОЛЬШОЙ ДАТАФРЕЙМ, ПОЛУЧАЯ НА ВХОД ДВА СЛОВА И МЕТКУ РИФМКЮТСЯ ИЛИ НЕТ. ВОЗВРАЩАЕТ 
# ДАТАФРЕЙМ РАЗМЕРОМ N НА 46, ГДЕ N - КОЛЛИЧЕСТВО РИФМОВАННЫХ + НЕ РИФМРВАННЫХ ПАР СЛОВ

# НЕ ЗАПУСКАТЬ!!! ГРУЗИТСЯ ДОЛГОООО

# with open('dataslittle.csv', 'r', encoding = 'utf-8') as f:
#     data = f.readlines()

# dataf = pd.DataFrame(columns = range(0, 46))
# count_col = 1 

# for stroka in data:
#     three_words = stroka.replace('\n', '').split(';')
#     list_big = []
#     list_two_w_to_copare = []
#     for word in three_words:
#         if three_words.index(word) == 2:
#             pass
#         else:
#             seg = make_segments_for_word(make_trans(word + '#'))
#             list_two_w_to_copare.append(create_matrix_priz(list_priznaki, seg))
#     if len(list_two_w_to_copare[0]) < 3  or len(list_two_w_to_copare[1]) < 3:
#         pass
#     else:
#         print(count_col)
#         egal = matr_egal(list_two_w_to_copare[0], list_two_w_to_copare[1])
#         dataf.loc[count_col, 0:44] = egal
#         dataf.loc[count_col, 45] = three_words[-1]
#     count_col += 1
# dataf.to_csv('data_vectots.csv')

In [0]:
dataf.shape

(97128, 46)